In [ ]:
#Ngoc Tuan HOANG - s147568
#Thesis topic: Heuristic solution approach for the Capacitated Lot-sizing and Scheduling Problem with 
#Sequence-Dependent Setup Times and Costs

#This code is used to find Lower bound of the problem.

In [1]:
using JuMP          
using Gurobi 

In [2]:
#In this box, three parameters need to be fulfilled. 
#inst: number of the instance being tested
#N: number of products
#T: number of time periods

#Information about instances can be found in the Appendix 2 at the end of the thesis.

inst = 8
Data = readdlm("Instance$(inst).txt")
N=5
T=20

20

In [3]:
#These lines of code are used to read all deterministic parameters (input data).        

d=cell(N,T)
for i=1:N
    for j=1:T
        d[i,j]=Data[i,j]
    end
end
r=cell(N,T)
for i=1:N
    for j=1:T
        r[i,j]=Data[N+i,j]
    end
end
hn=cell(N,T)
for i=1:N
    for j=1:T
        hn[i,j]=Data[2*N+i,j]
    end
end
hr=cell(N,T)
for i=1:N
    for j=1:T
        hr[i,j]=Data[3*N+i,j]
    end
end
an=cell(N,T)
for i=1:N
    for j=1:T
        an[i,j]=Data[4*N+i,j]
    end
end
ar=cell(N,T)
for i=1:N
    for j=1:T
        ar[i,j]=Data[5*N+i,j]
    end
end
p=cell(N,T)
for i=1:N
    for j=1:T
        p[i,j]=Data[6*N+i,j]
    end
end
time=cell(N,N,T)
for t=1:T
    time[:,:,t]=Data[7*N+((t-1)*N+1):7*N+((t-1)*N+N),1:N]
end
cost=cell(N,N,T)
for t=1:T
    cost[:,:,t]=Data[7*N+N*T+((t-1)*N+1):7*N+N*T+((t-1)*N+N),1:N]
end
cap=cell(1,T)
for j=1:T
    cap[1,j]=Data[7*N+2*N*T+1,j]
end
b=cell(N,1)
for i=1:N
    b[i,1]=Data[7*N+2*N*T+1+i,1]
end
jo=Data[8*N+2*N*T+2,1]

3

In [4]:
#The model and variables are defined.

m=Model(solver=GurobiSolver())
@defVar(m,XN[i=1:N,t=1:T,s=1:N]>=0)
@defVar(m,XR[i=1:N,t=1:T,s=1:N]>=0)
@defVar(m,InvN[i=1:N,t=1:T]>=0)
@defVar(m,InvR[i=1:N,t=1:T]>=0)
@defVar(m,B[i=1:N,t=1:T]>=0)
@defVar(m,0<=Y[i=1:N,j=1:N,t=1:T,s=1:N]<=1)

5x5x20x5 Array{JuMP.Variable,4}:
[:, :, 1, 1] =
 Y[1,1,1,1]  Y[1,2,1,1]  Y[1,3,1,1]  Y[1,4,1,1]  Y[1,5,1,1]
 Y[2,1,1,1]  Y[2,2,1,1]  Y[2,3,1,1]  Y[2,4,1,1]  Y[2,5,1,1]
 Y[3,1,1,1]  Y[3,2,1,1]  Y[3,3,1,1]  Y[3,4,1,1]  Y[3,5,1,1]
 Y[4,1,1,1]  Y[4,2,1,1]  Y[4,3,1,1]  Y[4,4,1,1]  Y[4,5,1,1]
 Y[5,1,1,1]  Y[5,2,1,1]  Y[5,3,1,1]  Y[5,4,1,1]  Y[5,5,1,1]

[:, :, 2, 1] =
 Y[1,1,2,1]  Y[1,2,2,1]  Y[1,3,2,1]  Y[1,4,2,1]  Y[1,5,2,1]
 Y[2,1,2,1]  Y[2,2,2,1]  Y[2,3,2,1]  Y[2,4,2,1]  Y[2,5,2,1]
 Y[3,1,2,1]  Y[3,2,2,1]  Y[3,3,2,1]  Y[3,4,2,1]  Y[3,5,2,1]
 Y[4,1,2,1]  Y[4,2,2,1]  Y[4,3,2,1]  Y[4,4,2,1]  Y[4,5,2,1]
 Y[5,1,2,1]  Y[5,2,2,1]  Y[5,3,2,1]  Y[5,4,2,1]  Y[5,5,2,1]

[:, :, 3, 1] =
 Y[1,1,3,1]  Y[1,2,3,1]  Y[1,3,3,1]  Y[1,4,3,1]  Y[1,5,3,1]
 Y[2,1,3,1]  Y[2,2,3,1]  Y[2,3,3,1]  Y[2,4,3,1]  Y[2,5,3,1]
 Y[3,1,3,1]  Y[3,2,3,1]  Y[3,3,3,1]  Y[3,4,3,1]  Y[3,5,3,1]
 Y[4,1,3,1]  Y[4,2,3,1]  Y[4,3,3,1]  Y[4,4,3,1]  Y[4,5,3,1]
 Y[5,1,3,1]  Y[5,2,3,1]  Y[5,3,3,1]  Y[5,4,3,1]  Y[5,5,3,1]

...

[:, :, 18, 1] 

In [5]:
#Objective function

@setObjective(m,Min,sum{an[i,t]*XN[i,t,s]+ar[i,t]*XR[i,t,s],i=1:N,t=1:T,s=1:N}
+sum{hn[i,t]*InvN[i,t]+hr[i,t]*InvR[i,t],i=1:N,t=1:T}
+sum{cost[i,j,t]*Y[i,j,t,s],i=1:N,j=1:N,t=1:T,s=1:N;j !=i}+sum{b[i]*B[i,t],i=1:N,t=1:T})

#Constraint (1.a)

for i=1:N
    for t=2:T             #inventory of finished products
        @addConstraint(m,InvN[i,t]-B[i,t]==InvN[i,t-1]-B[i,t-1]+sum{XN[i,t,s]+XR[i,t,s],s=1:N}-d[i,t])  
    end
end
for i=1:N
    for t=1            #inventory of finished products at 1st week 
        @addConstraint(m,InvN[i,t]-B[i,t]==sum{XN[i,t,s]+XR[i,t,s],s=1:N}-d[i,t])
    end
end

#Constraint (1.b)

for i=1:N
    for t=2:T             #inventory of return products
        @addConstraint(m,InvR[i,t]==InvR[i,t-1]+r[i,t]-sum{XR[i,t,s],s=1:N})
    end
end
for i=1:N
    for t=1            #inventory of return products at 1st week 
        @addConstraint(m,InvR[i,t]==r[i,t]-sum{XR[i,t,s],s=1:N})
    end
end


#Constraint (2)

for i=1:N       #backorder at the end
    for t=T     
        @addConstraint(m,B[i,t]==0)
    end
end

#Constraint (3) - Capacity constraint

for t=1:T       
        @addConstraint(m,sum{p[i,t]*(XN[i,t,s]+XR[i,t,s]),i=1:N,s=1:N}+
                sum{time[i,j,t]*Y[i,j,t,s],i=1:N,j=1:N,s=1:N}<=cap[t])
end

#Constraint (4)

for j=1:N
    for i=1:N
        if j!=jo
            @addConstraint(m,Y[j,i,1,1]==0)
        end
    end
end    

#Constraint (5)

@addConstraint(m,sum{Y[jo,i,1,1],i=1:N}==1)

#Constraint (6)

for i=1:N
    for t=1:T
        for s=1:(N-1)
             @addConstraint(m,sum{Y[j,i,t,s],j=1:N}==sum{Y[i,k,t,s+1],k=1:N})    
        end
    end
end

#Constraint (7)

for i=1:N
    for t=2:T
        @addConstraint(m,sum{Y[j,i,t-1,N],j=1:N}==sum{Y[i,k,t,1],k=1:N})    
    end
end

#Constraint (8)

for i=1:N
    for t=1:T
        for s=1:N
            @addConstraint(m,XN[i,t,s]+XR[i,t,s]<=sum(d)*sum{Y[j,i,t,s],j=1:N})    
        end
    end
end


#Constraint (9)

for t=1:T
    for s=1:N
        @addConstraint(m,sum{Y[j,i,t,s],i=1:N,j=1:N}<=1)
    end
end

#Constraint (10)

for j=1:N
    for t=1:T 
        @addConstraint(m,sum{Y[i,j,t,s],i=1:N,s=1:N}<=1)    
    end
end

show(m)

Minimization problem with:
 * 1441 linear constraints
 * 3800 variables
Solver set to Gurobi.Gurobi

In [6]:
#Solving the relaxed problem

tic()
status=solve(m)
toc()

Optimize a model with 1441 rows, 3800 columns and 18565 nonzeros
Coefficient statistics:
  Matrix range    [1e+00, 2e+04]
  Objective range [1e+00, 6e+01]
  Bounds range    [1e+00, 1e+00]
  RHS range       [1e+00, 2e+04]
Presolve removed 27 rows and 26 columns
Presolve time: 0.01s
Presolved: 1414 rows, 3774 columns, 18438 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.4579000e+02   2.541000e+04   0.000000e+00      0s
     888    3.2259097e+05   0.000000e+00   0.000000e+00      0s

Solved in 888 iterations and 0.03 seconds
Optimal objective  3.225909738e+05
elapsed time: 

2.883608516

2.883608516 seconds
